# Saved States Overview

In [1]:
from aquawar_game import game
save_dir = "saves/test_fix/game/"

In [2]:
import os
pkl_files = []
for file in os.listdir(save_dir):
    if file.endswith(".pkl"):
        pkl_files.append(file)

print("Found pkl files:", pkl_files)

Found pkl files: ['turn_000.pkl', 'latest.pkl', 'turn_001.pkl']


In [ ]:
import pickle
import json
for file in pkl_files:
    with open(os.path.join(save_dir, file), "rb") as f:
        state = pickle.load(f)
        print(f"Loaded state from {file}:")
        print(json.dumps(state, indent=2))

Loaded state from turn_000.pkl:
dict_keys(['state', 'version'])
{
  "state": {
    "players": [
      {
        "name": "Test1",
        "roster": [
          "Sunfish",
          "Sea Wolf",
          "Manta Ray",
          "Sea Turtle",
          "Octopus",
          "Great White Shark",
          "Hammerhead Shark",
          "Clownfish"
        ],
        "team": {
          "fish": [
            {
              "name": "Archerfish",
              "hp": 400,
              "atk": 100,
              "revealed": false,
              "buffs": [],
              "shields": 0,
              "dodge_chance": 0.0,
              "used_active_count": 0,
              "mimic_source": null
            },
            {
              "name": "Pufferfish",
              "hp": 400,
              "atk": 100,
              "revealed": false,
              "buffs": [],
              "shields": 0,
              "dodge_chance": 0.0,
              "used_active_count": 0,
              "mimic_source": null